<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>GPT Models</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 

import openai
from openai import OpenAI

import termcolor
from termcolor import colored

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 2f98fe4b5aabf6921b7e3b8fba71f74cb0ebead8

matplotlib: 3.8.0
tqdm      : 4.66.4
watermark : 2.4.3
numpy     : 1.26.4
termcolor : 2.4.0
openai    : 1.30.5
json      : 2.0.9
pandas    : 2.1.4



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the client

In [5]:
client = OpenAI()

We start by getting a list of supported models.

In [6]:
model_list = json.loads(client.models.list().json())["data"]

In total we have 32 models

In [7]:
len(model_list)

32

Along with some information about each model...

In [8]:
model_list[:3]

[{'id': 'dall-e-3',
  'created': 1698785189,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'whisper-1',
  'created': 1677532384,
  'object': 'model',
  'owned_by': 'openai-internal'},
 {'id': 'gpt-4-turbo-2024-04-09',
  'created': 1712601677,
  'object': 'model',
  'owned_by': 'system'}]

But let's just get a list of model names

In [9]:
print("\n".join(sorted([model["id"] for model in model_list])))

babbage-002
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-0301
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini
gpt-4o-mini-2024-07-18
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [10]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

Which produces a response object

In [11]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a named tuple

The model answer can be found in the "message" dictionary inside the "choices" list

In [11]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's weakness is Kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to Kryptonite weakens Superman and can potentially kill him.", role='assistant', function_call=None, tool_calls=None))

In [12]:
response.choices[0].message.content

"Superman's weakness is Kryptonite, a mineral from his home planet of Krypton that can weaken and harm him. Additionally, exposure to red solar radiation can also weaken him."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [13]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "What are the different kinds of Kryptonite?"},
    ],
    n=3
)

CPU times: user 11.2 ms, sys: 2.28 ms, total: 13.5 ms
Wall time: 4.29 s


And we can access each of the answers individually int he choices list

In [14]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
There are numerous variations of Kryptonite in the DC Comics universe, each with different effects on Kryptonians, particularly Superman. Some of the most well-known types of Kryptonite include:

1. Green Kryptonite: This is the most common form of Kryptonite and has the ability to weaken and harm Kryptonians, especially Superman. Exposure to green Kryptonite can leave them vulnerable, powerless, or even fatally poisoned.

2. Red Kryptonite: This type of Kryptonite has unpredictable effects on Kryptonians, often altering their behavior or physical appearance temporarily. It can also cause psychological changes, making the individual act out of character.

3. Blue Kryptonite: Blue Kryptonite is typically harmless to Kryptonians, but it has been known to have negative effects on Bizarro, a flawed duplicate of Superman.

4. Gold Kryptonite: Gold Kryptonite has the ability to permanently strip a Kryptonian of their powers, rendering them powerless for the rest of their lives.

5.

In [16]:
response.usage

CompletionUsage(completion_tokens=859, prompt_tokens=17, total_tokens=876)

# Temperature

In [23]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a dad joke"},
    ],
    temperature=1.8
)

CPU times: user 9.35 ms, sys: 1.93 ms, total: 11.3 ms
Wall time: 667 ms


In [24]:
print(response.choices[0].message.content)

Why was the math book sad?

Because it had too many problems.


In [21]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Tell me a dad joke"},
    ],
    temperature=0
)

CPU times: user 12.3 ms, sys: 3.11 ms, total: 15.4 ms
Wall time: 773 ms


In [22]:
print(response.choices[0].message.content)

Why couldn't the bicycle stand up by itself?

Because it was two tired!


# Function Calls

In [25]:
def chat(messages, functions):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        # Define the functions the model is allowed to use
        functions=functions
    )
    
    return response

In [32]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
#         print(message)
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and message['function_call']:
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message['role']]))
        elif message["role"] == "assistant" and not message['function_call']:
            print(colored(f"assistant: {message['content']}\n", role_to_color[message['role']]))
        elif message["role"] == "function":
            print(colored(f"function ({message.name}): {message.content}\n", role_to_color[message.role]))


Let's create some function specifications to interface with a hypothetical weather API. We'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [28]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [29]:
messages = []

messages.append(
    {"role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    })

messages.append(
    {"role": "user", 
     "content": "What's the weather like today"
    })

In [30]:
chat_response = chat(messages, functions=functions)

assistant_message = chat_response.choices[0].message

messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

In [33]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, can you please provide me with the location for which you want to know the weather?



Once we provide the missing information, it will generate the appropriate function arguments for us.

In [34]:
messages.append(
    {"role": "user", 
     "content": "I'm in New York, NY."
    })

In [35]:
chat_response = chat(messages, functions=functions)

assistant_message = chat_response.choices[0].message

messages.append({
 "role":  assistant_message.role,
 "content":  assistant_message.content,
 "function_call":  assistant_message.function_call,
})

In [36]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

assistant: Sure, can you please provide me with the location for which you want to know the weather?

user: I'm in New York, NY.

assistant: FunctionCall(arguments='{"location":"New York, NY","format":"celsius"}', name='get_current_weather')



## Few-shot prompting

We can also provide several examples of mappings between input and output.

In [37]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful, pattern-following assistant."},
        {"role": "user", "content": "Help me translate the following corporate jargon into plain English."},
        {"role": "assistant", "content": "Sure, I'd be happy to!"},
        {"role": "user", "content": "New synergies will help drive top-line growth."},
        {"role": "assistant", "content": "Things working well together will increase revenue."},
        {"role": "user", "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage."},
        {"role": "assistant", "content": "Let's talk later when we're less busy about how to do better."},
        {"role": "user", "content": "This late pivot means we don't have time to boil the ocean for the client deliverable."},
    ],
    temperature=0,
)

print(response.choices[0].message.content)

This last-minute change means we can't spend excessive time on the client's project.


# Formatted output

In [38]:
%%time
userInput = "blueberry pancakes"

prompt = """return a recipe for %s.
        Provide your response as a JSON object with the following schema:
        {"dish": "%s", "ingredients": ["", "", ...],
        "instructions": ["", "", ... ]}""" % (userInput, userInput)

response = client.chat.completions.create(
          model = "gpt-3.5-turbo",
          messages = [
            { "role": "system", "content": "You are a helpful recipe assistant."},
            { "role": "user",   "content": prompt }
          ]
)

CPU times: user 10.8 ms, sys: 1.86 ms, total: 12.7 ms
Wall time: 3.81 s


In [39]:
json_output = response.choices[0].message.content

In [40]:
output = json.loads(json_output)

In [43]:
pprint(output)

{'dish': 'blueberry pancakes',
 'ingredients': ['1 cup all-purpose flour',
                 '2 tbsp sugar',
                 '1 tbsp baking powder',
                 '1/2 tsp salt',
                 '1 cup milk',
                 '1 large egg',
                 '2 tbsp unsalted butter, melted',
                 '1 cup fresh blueberries',
                 'Butter or oil for cooking'],
 'instructions': ['In a bowl, whisk together the flour, sugar, baking powder, '
                  'and salt.',
                  'In another bowl, mix the milk, egg, and melted butter.',
                  'Combine the wet and dry ingredients, then gently fold in '
                  'the blueberries.',
                  'Heat a skillet or griddle over medium heat and grease with '
                  'butter or oil.',
                  'Pour 1/4 cup of batter onto the skillet for each pancake.',
                  'Cook until bubbles form on the surface, then flip and cook '
                  'until golden bro

In [44]:
output["ingredients"]

['1 cup all-purpose flour',
 '2 tbsp sugar',
 '1 tbsp baking powder',
 '1/2 tsp salt',
 '1 cup milk',
 '1 large egg',
 '2 tbsp unsalted butter, melted',
 '1 cup fresh blueberries',
 'Butter or oil for cooking']

In [45]:
output["instructions"]

['In a bowl, whisk together the flour, sugar, baking powder, and salt.',
 'In another bowl, mix the milk, egg, and melted butter.',
 'Combine the wet and dry ingredients, then gently fold in the blueberries.',
 'Heat a skillet or griddle over medium heat and grease with butter or oil.',
 'Pour 1/4 cup of batter onto the skillet for each pancake.',
 'Cook until bubbles form on the surface, then flip and cook until golden brown.',
 'Repeat with the remaining batter.',
 'Serve the blueberry pancakes warm with maple syrup and extra blueberries.']

# Translation

In [46]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": "You're a professional English-Italian translator."}, 
              {"role": "user", "content": "Translate 'Be the change that you wish to see in the world.' into Italian"}],
    temperature=0,
)

In [47]:
response.choices[0].message.content

'"Sii il cambiamento che desideri vedere nel mondo."'

# Process unstructured information

Inspired by https://platform.openai.com/examples/default-parse-data

In [49]:
prompt = """There are many fruits that were found on the recently discovered planet Goocrux. 
There are neoskizzles that grow there, which are purple and taste like candy. There are also 
loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits 
are a bright green color and are more savory than sweet. There are also plenty of loopnovas which 
are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which 
have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."""

In [50]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", 
         "content": "You will be provided with unstructured data, and your task is to parse it into CSV format."}, 
        {"role": "user", 
         "content": prompt}],
    temperature=0,
)

In [51]:
print(response.choices[0].message.content)

Fruit,Color,Flavor
neoskizzles,purple,candy
loheckles,grayish blue,tart
pounits,bright green,savory
loopnovas,neon pink,cotton candy
glowls,pale orange,sour and bitter


In [52]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "system", "content": """
            Read this paragraph 
            
            `%s` 
            
            and use it to answer some questions.""" % prompt}, 
              {"role": "user", "content": "What are pounits?"}],
    temperature=0,
)

In [53]:
print(response.choices[0].message.content)

Pounits are bright green fruits that are more savory than sweet.


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>